In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [15]:
file_path = "../files/MEPC 80차 결과 보고서_송부 20230713_한국해운협회 해무팀.pdf"

In [16]:
llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader(file_path)

In [5]:
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [10]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)
chain.run("문서를 요약 해줘")

'위 문서는 선박의 선박평형수관리협약과 벌크선 CII 산정에 대한 논의 내용을 다루고 있습니다. 벌크선에 대한 CII 산정 문제점을 해결하기 위해 현장 협의가 이뤄졌고, 2023년에 의제문서 작성을 위한 협의를 통해 MEPC 81차에 제출할 계획입니다. 또한, 중장기 조치 도입과 관련해 IMO와 EU 규제 중복사항에 대한 의견조사 결과가 확인되었으며, 관리행위에 대한 코드화 구조 개정과 평형수 기록부 작성지침 개정안이 논의되었습니다. 최종적으로, 수질악조건에서 운항하는 선박에 대한 선박평형수관리협약의 적용에 관한 임시 지침서의 채택이 이루어졌습니다.'

In [17]:
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [18]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)
chain.run("문서를 요약 해줘")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-WaJwtqxgWmREYBrgBhUgCiu3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-WaJwtqxgWmREYBrgBhUgCiu3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [8]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)
chain.run("문서를 요약 해줘")

/home/lab021/yhkim2/fullstack-gpt/venv/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-WaJwtqxgWmREYBrgBhUgCiu3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-WaJwtqxgWmREYBrgBhUgCiu3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.